## **Key-50. Оценка продуктовой фичи**
<font size="5" color="red"><b>DAPR-1016</b> Pidenko Dmitry</font>

### **Легенда**
Клининговая компания. Одна из продуктовых команд для своего сайта подготовила новую фичу, а именно контентную ленту в которой будет много различных статей, видео, лайфхаков. Дата релиза 1 июня, на календаре 31 мая. Необходимо расчитать несколько метрик  до и после внедрения фичи, для того что бы понять целесообразна ли она.

### **ЗАДАНИЕ**
1.Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.

2.Сформировать выводы по анализу.

### **Подготовка данных**

In [120]:
#Выгружаем нужные библиотеки 
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

In [121]:
#Конструкция pd.DataFrame(getRegisteredUsersCount()) позволяет выполнить функцию запроса, а затем записать полученные данные в датафрейм.
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
registered_users_count = pd.DataFrame(getRegisteredUsersCount())
#Преобразуем столбец registration_date к формату datetime
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

In [122]:
registered_users_count.head()

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266
3,2019-05-04,8639
4,2019-05-05,9787


In [123]:
#Создадим функцию getActiveUsersCountWithCohorts() и запишем данные, полученные с помощью этой функции, в 
# датафрейм active_users_count_with_cohorts
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
#Преобразуем столбцы 'activity_date','registration_date' к формату datetime
active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())
for i in ['activity_date','registration_date']:
    active_users_count_with_cohorts[i] = pd.to_datetime(active_users_count_with_cohorts[i])


In [124]:
active_users_count_with_cohorts.head()

,activity_date,registration_date,active_users_count
0,2019-05-01,2019-05-01,2590
1,2019-05-02,2019-05-01,2146
2,2019-05-02,2019-05-02,2049
3,2019-05-03,2019-05-01,1637
4,2019-05-03,2019-05-02,1817


In [125]:
#Соединяем таблицы
retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')

In [126]:
#В датафрейме retention_table нам нужно определить, какое количество дней прошло с момента регистрации до определённого дня активности.
#Создаем дополнителбный столбец lifetime
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
#Но timedelta при вычитании одной даты из другой будет подсчитана в днях. Это видно по типу данных в колонке timedelta64[ns].
#Нам нужно получить целое количество дней, поэтому значения в колонке с типом timedelta мы разделим на константу np.timedelta64(1,'D'):
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


In [127]:
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


In [128]:
#Создадим данные по доходам
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())
#Преобразуем столбцы 'date','registration_date' к формату datetime
for i in ['date','registration_date']:
    revenue[i] = pd.to_datetime(revenue[i])

In [129]:
#Переименовываем столбец 'date' аналогичному датафрейму retention_table, для того чтобы присоединить
revenue = revenue.rename(columns={'date':'activity_date'})
#Соединяем таблицы
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
#Смотрим, что получилось
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672.0,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164.0,533.608


In [130]:
#Заменяем пустые строки нулями, в столбцах 'revenue','users_count_with_revenue'
for i in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[i] = retention_table_with_revenue[i].fillna(0)
#Приводм столбец 'users_count_with_revenue' к числовому типу 
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608


### Данные подготовленны, можем выполнять расчеты. Фильтруем таблицу по retention 7 дня для пользователей, привлеченных до релиза (май 2019)

In [131]:
table_7day_may = retention_table_with_revenue[retention_table_with_revenue['lifetime'] == 7]\
[retention_table_with_revenue['registration_date'] >= '2019-05-01'][retention_table_with_revenue['registration_date'] < '2019-06-01']
table_7day_may.head()

C:\Users\dmitr\AppData\Local\Temp\ipykernel_6700\3972686221.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_7day_may = retention_table_with_revenue[retention_table_with_revenue['lifetime'] == 7]\
C:\Users\dmitr\AppData\Local\Temp\ipykernel_6700\3972686221.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_7day_may = retention_table_with_revenue[retention_table_with_revenue['lifetime'] == 7]\


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
28,2019-05-08,2019-05-01,953,6516,7,616,63.9066
37,2019-05-09,2019-05-02,745,5309,7,490,313.7230
47,2019-05-10,2019-05-03,895,6266,7,571,110.7260
58,2019-05-11,2019-05-04,1186,8639,7,750,124.8590
70,2019-05-12,2019-05-05,1358,9787,7,885,174.9240


### Выполняем расчет retention 7 дня до релиза

In [132]:
may_7day = table_7day_may['active_users_count'].sum() / table_7day_may['registered_users_count'].sum()
print(f'retention  7 дня до релиза, составляет: {round(may_7day * 100, 1)}%')

retention  7 дня до релиза, составляет: 12.9%


### Сформируем retention 7 дня для июня после релиза 

In [133]:
table_7day_june = retention_table_with_revenue[retention_table_with_revenue['lifetime'] == 7]\
[retention_table_with_revenue['registration_date'] >= '2019-06-01'][retention_table_with_revenue['registration_date'] < '2019-07-01']
table_7day_june.head()

C:\Users\dmitr\AppData\Local\Temp\ipykernel_6700\4039966676.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_7day_june = retention_table_with_revenue[retention_table_with_revenue['lifetime'] == 7]\
C:\Users\dmitr\AppData\Local\Temp\ipykernel_6700\4039966676.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_7day_june = retention_table_with_revenue[retention_table_with_revenue['lifetime'] == 7]\


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
736,2019-06-08,2019-06-01,545,4833,7,365,176.3160
767,2019-06-09,2019-06-02,683,5255,7,461,193.8390
798,2019-06-10,2019-06-03,484,4193,7,320,27.7548
829,2019-06-11,2019-06-04,511,4194,7,320,54.9610
860,2019-06-12,2019-06-05,472,3998,7,301,63.1554


### Произведем расчет 7 дня после релиза

In [134]:
june_7day = table_7day_june['active_users_count'].sum() / table_7day_june['registered_users_count'].sum()
print(f'retention 7 дня после релиза, составляет: {round(june_7day * 100, 1)}%')

retention 7 дня после релиза, составляет: 12.6%


In [135]:
#Посмотрим, увеличился или уменьшился retention после релиза
difference = (may_7day - june_7day) / may_7day  * 100
print(f'retention в июне уменьшился на: {round(difference,1)}%')

retention в июне уменьшился на: 2.9%


### Далее расчитаем ARPPU мая до релиза и июнь после релиза, (считаем средний доход именно на платящих клиентов, поэтому делить будем на 'users_count_with_revenue')

In [136]:
may_ARPPU = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>='2019-05-01']\
    [retention_table_with_revenue['registration_date']<'2019-06-01'].sum() / retention_table_with_revenue['users_count_with_revenue']\
        [retention_table_with_revenue['registration_date']>='2019-05-01'][retention_table_with_revenue['registration_date']<'2019-06-01'].sum()
print(f'ARPPU май до релиза, составил: = {round(may_ARPPU, 2)}Р')


ARPPU май до релиза, составил: = 0.31Р


### ARPPU за июнь после релиза

In [137]:
june_ARPPU = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>='2019-06-01']\
    [retention_table_with_revenue['registration_date']<'2019-07-01'].sum() / retention_table_with_revenue['users_count_with_revenue']\
        [retention_table_with_revenue['registration_date']>='2019-06-01'][retention_table_with_revenue['registration_date']<'2019-07-01'].sum()
print(f'ARPPU июнь после релиза, составил: = {round(june_ARPPU, 2)}Р')

ARPPU июнь после релиза, составил: = 0.39Р


In [138]:
#Глянем на разницу ARPPU до релиза и после:
difference_ARPPU = abs(may_ARPPU - june_ARPPU) / may_ARPPU * 100
print(f'ARPPU в июне (после релиза) значительно увеличился на: {round(difference_ARPPU,1)}%')

ARPPU в июне (после релиза) значительно увеличился на: 28.4%


## ВЫВОД:

1.При расчете метрики RETENTION RATE 7-ого дня до релиза и после, можно наблюдать не значительную разницу. В июне (после релиза), возвращаемость пользователей снизилась примерно на 3%.

2.А вот при сравнении дохода (метрики - ARPPU), после релиза новой фичи в июне, доход значительно увеличился почти на 30%.

Отсюда можно заключить, что продуктовый отдел поработал на славу. Пользователю дали то, что он хотел, возможно даже сам того не подозревая.

